In [1]:
import pandas as pd

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad de eventos total

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

In [9]:
features_df.head(3)

,person,cant_eventos_totales
0,4886f805,9
1,ad93850f,65
2,0297fc1e,567


## Cantidad de dias en los que entra

In [10]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['dia'] = events_df['timestamp'].dt.day
events_df['hora'] = events_df['timestamp'].dt.hour
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [11]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,dia,hora,weekday
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0,Friday
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0,Friday
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0,Friday
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0,Friday
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0,Friday


In [12]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct.head()

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
person,,,,,,,
0008ed71,0,0,0,0,6,0,0
00091926,86,32,32,131,84,43,40
00091a7a,0,10,0,0,0,0,0
000ba417,0,0,59,0,147,0,0
000c79fe,0,0,0,0,0,17,0


In [13]:
features_df = features_df.join(ct, on='person')

## Quincenas

In [14]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [15]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [16]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [17]:
features_df = features_df.join(df, on='person')

## Mean encoding channel 

In [18]:
#Mean encoding

labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [19]:
#Separo en train y to_predict
mean_encoding = events_df.merge(labels_df, on='person', how='right')
mean_encoding.shape

(1171886, 38)

In [20]:
me = mean_encoding.groupby(['channel']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_channel'}))
me

label,mean_enc_channel
channel,
Direct,0.112726
Email,0.107784
Organic,0.092337
Paid,0.066072
Referral,0.101998
Social,0.076923
Unknown,NaN


In [21]:
events_df = events_df.join(me, on='channel')

In [ ]:
#todo['me_channel'] = events_df.mean_enc_channel

In [22]:
events_df.channel.value_counts()

Paid        91753
Direct      50675
Referral    33990
Organic     25867
Social       1406
Email         371
Unknown         7
Name: channel, dtype: int64

In [23]:
events_df.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,q2,q3,q4,q5,q6,q7,q8,q9,q10,mean_enc_channel
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,False,False,False,False,False,False,False,False,True,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,False,False,False,False,False,False,False,True,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,False,False,False,False,False,False,False,False,True,NaN


In [ ]:
events_df.channel.value_counts()

In [24]:
events_df.mean_enc_channel.value_counts()

0.066072    91753
0.112726    50675
0.101998    33990
0.092337    25867
0.076923     1406
0.107784      371
Name: mean_enc_channel, dtype: int64

In [25]:
df = events_df.groupby(['person']).mean_enc_channel.mean()
df.head(3)

person
0008ed71    0.101998
00091926    0.101433
00091a7a    0.066072
Name: mean_enc_channel, dtype: float64

In [26]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

## Mean encoding storage

In [27]:
me = mean_encoding.groupby(['storage']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_storage'}))
me.head()

label,mean_enc_storage
storage,
128GB,0.078651
16GB,0.082392
256GB,0.064498
32GB,0.085803
4GB,0.084156


In [ ]:
events_df.storage.value_counts()

In [28]:
events_df = events_df.join(me, on='storage')

In [ ]:
#todo['me_storage'] = events_df.mean_enc_storage

In [ ]:
events_df.mean_enc_storage.value_counts()

In [29]:
df = events_df.groupby(['person']).mean_enc_storage.mean()
df.head(3)

person
0008ed71    0.083973
00091926    0.082193
00091a7a    0.080452
Name: mean_enc_storage, dtype: float64

In [30]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

## Mean encoding eventos

In [31]:
me = mean_encoding.groupby(['event']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_event'}))
me.head()

label,mean_enc_event
event,
ad campaign hit,0.064665
brand listing,0.105026
checkout,0.055226
conversion,0.122334
generic listing,0.095422


In [32]:
events_df.event.value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [33]:
events_df = events_df.join(me, on='event')

In [ ]:
#todo['me_event'] = events_df.mean_enc_event

In [34]:
events_df.mean_enc_event.value_counts()

0.085458    1248124
0.105026     216312
0.087264     204069
0.064665     191388
0.095422     160176
0.080361     130616
0.076132     106406
0.055226      65315
0.104064      11201
0.122334       7091
0.125290        983
Name: mean_enc_event, dtype: int64

In [35]:
df = events_df.groupby(['person']).mean_enc_event.mean()
df.head(3)

person
0008ed71    0.072605
00091926    0.085856
00091a7a    0.093343
Name: mean_enc_event, dtype: float64

In [36]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

In [ ]:
todo.head()

In [37]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 23)

In [ ]:
#Separo en train y to_predict
#train_df = todo.merge(labels_df, on='person', how='right')
#train_df.shape

In [38]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 22)

In [ ]:
#to_predict = todo[~todo.person.isin(labels_df.person)]
#to_predict.shape

In [39]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [40]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [41]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [42]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [43]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [44]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3, 0.5],
        'colsample_bylevel': [0.7, 0.8, 0.9],
        'max_depth': [3, 4, 5, 7, 8, 9, 11, 13, 15],
        'base_score': [0.1, 0.2, 0.3],
        'n_estimators': [35, 40, 50, 60, 70, 80, 90],
        'booster':['gbtree', 'dart']
        }

In [45]:
xg_gs = xgb.XGBRegressor(objective ='binary:logistic',
                        learning_rate = 0.2,
                        alpha = 20, eval_metric='auc')

In [46]:
folds = 10
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_gs, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, 
                                   cv=skf.split(train_df.drop(columns=['person']),labels), verbose=3, random_state=1001,return_train_score=True)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_df.drop(columns=['person']), labels)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   18.0s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:   32.2s finished



 Time taken: 0 hours 0 minutes and 34.73 seconds.


In [47]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([3.49346962, 2.30148392, 0.6566638 , 0.3620543 , 1.41905203,
       0.61193523, 2.16665533]), 'std_fit_time': array([0.12107328, 0.11677359, 0.06154419, 0.03217198, 0.10460295,
       0.04523012, 0.20903224]), 'mean_score_time': array([0.01918962, 0.01211605, 0.01065848, 0.01063838, 0.01728556,
       0.01334724, 0.01229758]), 'std_score_time': array([0.00664906, 0.00303659, 0.00235015, 0.00179799, 0.00860654,
       0.00146966, 0.00184732]), 'param_n_estimators': masked_array(data=[70, 70, 35, 40, 70, 50, 70],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 10, 1, 5, 5, 1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 11, 4, 4, 5, 7, 15],
             mask=[False, False, False, False, False, False, False],
       

In [48]:
preds_grid_search = random_search.predict(X_test)

In [49]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds_grid_search)

0.885299875855318

In [50]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [52]:
preds_posta

array([0.0150192 , 0.12940146, 0.09153673, ..., 0.06014185, 0.05194858,
       0.05194858], dtype=float32)

In [53]:
to_publish = pd.DataFrame()

In [54]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [56]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [57]:
to_publish.head()

,person,label
0,4886f805,0.015019
1,0297fc1e,0.129401
2,2d681dd8,0.091537
3,cccea85e,0.107137
4,4c8a8b93,0.039037


In [58]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [59]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [60]:
to_publish.to_csv('3_12_1.csv', index=False)